In [1]:
import sys
sys.path.insert(0, './src')  # 或者绝对路径：/full/path/to/your_project
from src.xmcdata import *
from gen_model import *
from verbalizer import *


In [2]:
data_dir = "./xmc-base/"
dataset_name = "eurlex-4k/"
data_dir = data_dir + dataset_name
model_names = [
        "google/flan-t5-base",  # T5-base
        "google/flan-t5-large",  # T5-large
        "google/flan-t5-xl",     # T5-xl
        "facebook/bart-base",    # BART
        "google/pegasus-xsum"    # Pegasus
    ]

In [ ]:


label_map = load_label_text_map(data_dir+"output-items.txt")

label_map = [v for k, v in sorted(label_map.items(), key=lambda x: x[0])]

pred_label_list = []
with open(data_dir+"gen_pred_"+model_names[1].split("/")[-1]+".txt","r") as r:
    for i in r:
        pred_label_list.append(i.strip(".").split(","))
replacer = LabelReplacementSystem(label_map,model_name='all-MiniLM-L6-v2')
replaced_result = replacer.replace_batch(pred_label_list)
final_labels = [result['final'] for result in replaced_result]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

标签库大小: 3956
使用模型: all-MiniLM-L6-v2
计算标签库embeddings...


Batches:   0%|          | 0/62 [00:00<?, ?it/s]

标签库embeddings计算完成!


处理标签列表: 100%|██████████| 3865/3865 [00:22<00:00, 172.55it/s]


In [6]:
replacer.get_summary_stats(replaced_result)

{'total_lists_processed': 3865,
 'total_labels_original': 19216,
 'total_labels_after_dedup': 19099,
 'deduplication_reduction': '0.6%',
 'total_replacements_made': 4332,
 'total_kept_from_library': 14767,
 'replacement_rate': '22.7%',
 'avg_similarity_score': '0.919',
 'min_similarity_score': '0.401',
 'max_similarity_score': '0.993'}

In [7]:
with open(data_dir+"test_replaced.txt", "w")as w:
    for i in final_labels:
        w.write(",".join(i)+"\n")



In [3]:
t5_config = T2TConfig(
        model_name=model_names[1],  # Change to other   models as needed
        dataset_dir=data_dir,
        output_dir=data_dir+"/outputs/"+model_names[1].split("/")[-1],
        prompt="Summarize this document by unstemmed keyphrases:",
        #task_prefix="classify:",  # T5-style task  prefix
        num_epochs=5,
        batch_size=4, # base-16, large-4, xl-2? for    24GB
        learning_rate= 2e-5,
        max_input_length=512,
        max_output_length=128,
        strict_tokenization=True,
        fp16=False, #t5 cannot use fp16
        bf16= True,
    )
model = UniversalT2TModel(t5_config)
model.load_local_model(model_path=data_dir+"/outputs/"+model_names[1].split("/")[-1])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loaded T5 tokenizer: google/flan-t5-large
Loading T5 model: google/flan-t5-large
Model loaded on cuda
Model type: T5
Model parameters: 783,150,080
Loading local model from ./xmc-base/eurlex-4k//outputs/flan-t5-large...
Local model loaded successfully from ./xmc-base/eurlex-4k//outputs/flan-t5-large


In [ ]:
model.predict_train_set(data_dir)

Loading tokenized data from cache:./xmc-base/eurlex-4k//train.pt
Loaded 15449 samples from cache
Loading tokenized data from cache:./xmc-base/eurlex-4k//test.pt
Loaded 3865 samples from cache


Generating with T5 Standards:  54%|█████▍    | 2083/3863 [36:23<31:45,  1.07s/it]  

In [ ]:
from src.ranker import *
ranker = DocumentLabelRanker()